In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
import matplotlib.pyplot as plt

In [2]:
qreg_q = QuantumRegister(3, 'q')
creg_b0 = ClassicalRegister(1, 'b0')
creg_b1 = ClassicalRegister(1, 'b1')
creg_aq = ClassicalRegister(1, 'aq')
circuit = QuantumCircuit(qreg_q, creg_b0, creg_b1, creg_aq)

circuit.h(qreg_q[0])
circuit.s(qreg_q[0])
circuit.t(qreg_q[0])
circuit.h(qreg_q[0])
circuit.h(qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[2])

circuit.barrier(qreg_q[0], qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.h(qreg_q[0])
circuit.measure(qreg_q[0], creg_b0[0])
circuit.measure(qreg_q[1], creg_b1[0])
circuit.barrier(qreg_q[0], qreg_q[1], qreg_q[2])

circuit.x(qreg_q[2]).c_if(creg_b1, 1)
circuit.z(qreg_q[2]).c_if(creg_b0, 1)
circuit.measure(qreg_q[2], creg_aq[0])

#drawing the circuit
circuit.draw('mpl').savefig('circuit.png')
# plt.savefig('circuit.png')

In [3]:
# Adding the transpiler to reduce the circuit to QASM instructions
# supported by the backend
from qiskit import transpile

# Use Aer's qasm_simulator
from qiskit.providers.aer import QasmSimulator

backend = QasmSimulator()

# First we have to transpile the quantum circuit
# to the low-level QASM instructions used by the
# backend
qc_compiled = transpile(circuit, backend)

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = backend.run(qc_compiled, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

In [4]:
counts = result_sim.get_counts(qc_compiled)
print(counts)

{'1 0 1': 227, '1 0 0': 212, '1 1 1': 231, '1 1 0': 210, '0 1 1': 43, '0 1 0': 31, '0 0 0': 25, '0 0 1': 45}


In [5]:
from qiskit.visualization import plot_histogram
plot_histogram(counts).savefig('hist.png')
# plt.savefig('hist.png')